In [7]:
import tfidf_matcher as tm
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

In [8]:
# Workstation Path
#nets_all_data = pd.read_stata('S:/Users/Hiwi/Maris/name_matching/Data/hq_all_maris.dta')
#nets_pub_data = pd.read_stata('S:/Users/Hiwi/Maris/name_matching/Data/hq_companies_public_maris.dta')
#compustat_data = pd.read_stata('S:/Users/Hiwi/Maris/name_matching/Data/compustat.dta')

# Mac Path
compustat_data = pd.read_stata('/Users/marisbuttmann/Desktop/HIWI_code/geocode_project/all_data/compustat.dta')
nets_pub_data = pd.read_stata('/Users/marisbuttmann/Desktop/HIWI_code/geocode_project/all_data/hq_companies_public_maris.dta')
#nets_all_data = pd.read_stata('/Users/marisbuttmann/Desktop/HIWI_code/geocode_project/all_data/hq_all_maris.dta')

In [9]:
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

def matching(compustat_names, nets_names):
    workers = 5
    results = []

    # Split nets_all_data into 4 pieces
    nets_splits = np.array_split(nets_names, workers)

    with ThreadPoolExecutor(max_workers=workers) as executor:
        # Submit a task for each split of nets_all_data
        futures = [executor.submit(tm.matcher, compustat_names, data_split, k_matches=3, ngram_length=3) for data_split in nets_splits]

        for future in as_completed(futures):
            try:
                results.append(future.result())
            except Exception as e:
                print(e)

    return results

In [13]:
compustat = compustat_data['hqcompany']
nets = nets_pub_data['hqcompany']
result = tm.matcher(compustat, nets)

In [23]:
result[result['Lookup 1 Confidence'] > 0.5].sort_values('Lookup 1 Confidence', ascending=False).sample(100)

,Original Name,Lookup 1,Lookup 1 Confidence,Lookup 1 Index,Lookup 2,Lookup 2 Confidence,Lookup 2 Index,Lookup 3,Lookup 3 Confidence,Lookup 3 Index,Lookup 4,Lookup 4 Confidence,Lookup 4 Index,Lookup 5,Lookup 5 Confidence,Lookup 5 Index
12583,TELENETICS CORP,TELENETICS CORPORATION ...,0.52,15134,ESCAGENETICS CORPORATION ...,0.33,25756,CELLCYTE GENETICS CORPORATION ...,0.33,25256,NEWLINK GENETICS CORPORATION ...,0.33,16129,CROWN ELECTROKINETICS CORP ...,0.31,16369
28317,ESTEE LAUDER COMPANIES INC,ESTEE LAUDER COMPANIES INC ...,0.75,536,GUARANTEE LIFE COMPANIES INC ...,0.37,9380,TRUSTEE LIFE INSURANCE COMPANY ...,0.27,23509,APPLETREE COMPANIES INC ...,0.25,14389,PIONEER COMPANIES INC ...,0.25,5507
31603,NIOCORP DEVELOPMENTS LTD,NIOCORP DEVELOPMENTS LTD ...,0.74,16302,TRI-NATIONAL DEVELOPMENT CORP ...,0.44,6955,GLOBAL ENERGY DEVELOPMENT LTD ...,0.44,25078,EDUCATIONAL DEVELOPMENT CORP ...,0.42,6057,TECHNOLOGY DEVELOPMENT CORP ...,0.42,10475
21355,FOOD INTEGRATED TECH INC,HEALTHCARE INTEGRATED TECH INC ...,0.53,15277,ACCESS INTEGRATED TECH INC ...,0.51,23656,ACCESS INTEGRATED TECHNOLOGIES ...,0.44,17654,INTEGRATED DEVICE TECH INC ...,0.44,5118,INTEGRATED TECHNOLOGY USA INC ...,0.43,25565
13295,FOREMOST CORP OF AMERICA,FOREMOST CORP OF AMERICA ...,0.69,13404,FOREMOST CORPORATION AMERICA ...,0.56,24339,TRANSPORT CORP OF AMERICA ...,0.38,7711,COGENERATION CORP OF AMERICA ...,0.38,21928,FIRST REPUBLIC CORP OF AMERICA ...,0.37,215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42907,BIODEXA PHARMACEUTICALS PLC,OSMOTICA PHARMACEUTICALS PLC ...,0.54,15885,ARMATA PHARMACEUTICALS INC ...,0.44,13879,BIOTA PHARMACEUTICALS INC ...,0.43,10528,CRINETICS PHARMACEUTICALS INC ...,0.43,25983,ALSERES PHARMACEUTICALS INC ...,0.43,21007
36145,PORTEC RAIL PRODUCTS INC,ABC RAIL PRODUCTS CORP DEL ...,0.54,10233,CRYSTAL PRODUCTS CORPORATION ...,0.35,26614,ARDEN INDUSTRIAL PRODUCTS INC ...,0.35,25605,AMERICAN EDUCATIONAL PRODUCTS ...,0.35,13693,LUTHER MEDICAL PRODUCTS INC ...,0.34,17378
2439,BLOCKBUSTER ENMNT CORP,BLOCKBUSTER ENTERTAINMENT CORP ...,0.58,10035,BLOCKBUSTER INC ...,0.40,730,BLOC DEVELOPMENT CORPORATION ...,0.18,13335,BLOCKCHAIN INDUSTRIES INC ...,0.17,11263,BLOCKCHAIN MOON ACQSITION CORP ...,0.16,19344
20018,BUSINESS FIRST BANCSHS INC,BUSINESS FIRST BANCSHARES INC ...,0.69,4174,FIRST BUSINESS FINCL SVCS INC ...,0.53,8928,AMERICAN BUSINESS FINCL SVCS ...,0.35,2196,SOUTHERN FIRST BANCSHARES INC ...,0.34,7667,RAINEN BUSINESS INTERIORS INC ...,0.34,24648
